<a href="https://colab.research.google.com/github/alik604/Notebooks/blob/master/amonalyDetection/Intrusion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intrusion Detection 

### Dataset from https://www.kaggle.com/what0919/intrusion-detection

### Sample code used: https://www.kaggle.com/nidhirastogi/intrusion-detection/data


In [0]:
%config IPCompleter.greedy=True
import pandas as pd
import seaborn as sns
import numpy as np
import re
import sklearn

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib as matplot
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [64]:
import os

os.environ['KAGGLE_USERNAME'] = "alik604" # username from the json file os.environ['KAGGLEKEY'] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx" # key from the json file
os.environ['KAGGLE_KEY'] = "1c6c5de0f6a4d1610a2f8f5ad6771bb7"

!mkdir /root/.kaggle
!echo '{"username":"alik604","key":"1c6c5de0f6a4d1610a2f8f5ad6771bb7"}' > /root/.kaggle/kaggle.json

!kaggle datasets download -d what0919/intrusion-detection
!ls

mkdir: cannot create directory ‘/root/.kaggle’: File exists
intrusion-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
intrusion-detection.zip  sample_data  test_data.csv  Train_data.csv


In [65]:
import zipfile

zip_ref = zipfile.ZipFile('intrusion-detection.zip', 'r')
zip_ref.extractall()
zip_ref.close()
!ls

intrusion-detection.zip  sample_data  test_data.csv  Train_data.csv


In [0]:
train = pd.read_csv('./Train_data.csv')
test = pd.read_csv('./test_data.csv')
test = test.drop('Unnamed: 0', axis=1)

### Reduce train size for faster trainin, remove when in production

In [0]:
# train=train.sample(frac =.50,random_state=1) # TODO 

### The following few cells are taken from the 'sample code'

In [68]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


set(list(train['xAttack']))
train['xAttack'] = le.fit_transform(train['xAttack'])
test['xAttack'] = le.fit_transform(test['xAttack'])

train['xAttack'].describe()

train['protocol_type'] = le.fit_transform(train['protocol_type'])
test['protocol_type'] = le.fit_transform(test['protocol_type'])

print('\n')
tmp = train.corr().abs().sort_values('xAttack')
garbage = tmp['xAttack'].nsmallest(5)

garbage
garbage = list(garbage.index) + ['num_outbound_cmds','is_host_login']



{'dos', 'normal', 'probe', 'r2l', 'u2r'}

count    125973.000000
mean          0.744985
std           0.653748
min           0.000000
25%           0.000000
50%           1.000000
75%           1.000000
max           4.000000
Name: xAttack, dtype: float64

is_host_login      0.001099
num_compromised    0.004279
num_root           0.005006
dst_bytes          0.008763
land               0.010021
Name: xAttack, dtype: float64

In [0]:
X_train = train.drop('xAttack', axis=1)
Y_train = train.loc[:,['xAttack']]
X_test = test.drop('xAttack', axis=1)
Y_test = test.loc[:,['xAttack']]

## Drop features with lowest STD

In [70]:
# df.ix[2,:]==0 # see where STD== 0, which means all values are the same... which is useless 
# train = train.drop('num_outbound_cmds', axis=1)
## bad sample code I fixed 

con_list = [
    'protocol_type', 'service', 'flag', 'land', 'logged_in', 'su_attempted',
    'is_host_login', 'is_guest_login'
]

df = X_train.drop(con_list, axis=1)

#drop n smallest std features
df = df.std(axis=0).to_frame()
tmp = df.nsmallest(5, columns=0)
tmp = list(
    tmp.transpose().columns)  #fuckin pandas.core.indexes.base.Index   -_-
#tmp = tmp.append('num_outbound_cmds')  # might not work...
tmp = set(tmp + garbage)
len(tmp)
tmp

10

{'dst_bytes',
 'is_host_login',
 'land',
 'num_compromised',
 'num_failed_logins',
 'num_outbound_cmds',
 'num_root',
 'num_shells',
 'root_shell',
 'urgent'}

In [0]:
from sklearn import linear_model

from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import IsolationForest

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [75]:
LR = linear_model.LinearRegression()
LR.fit(X_train, Y_train)
lr_score = LR.score(X_test, Y_test)
print('Linear regression processing ,,,')
print('Linear regression Score: %.2f %%' % lr_score)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Linear regression processing ,,,
Linear regression Score: 0.30 %


In [73]:
try:  #TODO
    X_train = X_train.drop(tmp,axis=1)
    X_test = X_test.drop(tmp,axis=1)
except:
    None
    
X_train.shape
X_test.shape

{'dst_bytes',
 'is_host_login',
 'land',
 'num_compromised',
 'num_failed_logins',
 'num_outbound_cmds',
 'num_root',
 'num_shells',
 'root_shell',
 'urgent'}

(125973, 31)

(10000, 31)

In [74]:
LR = linear_model.LinearRegression()
LR.fit(X_train, Y_train)
lr_score = LR.score(X_test, Y_test)
print('Linear regression processing ,,,')
print('Linear regression Score: %.2f %%' % lr_score)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Linear regression processing ,,,
Linear regression Score: 0.30 %


In [0]:
AB = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, learning_rate=1.0)
RF = RandomForestClassifier(n_estimators=10, criterion='entropy', max_features='auto', bootstrap=True)
ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False)
GB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200, max_features='auto')

In [0]:
# y_train = Y_train['xAttack'].ravel()
# x_train = X_train.values
# x_test = X_test.values

In [38]:
AB.fit(X_train, Y_train)
AB_feature = AB.feature_importances_
#AB_feature
ab_score = AB.score(X_test, Y_test)

print('AdaBoostClassifier processing ,,,')
print('AdaBoostClassifier Score: %.3f %%' % ab_score)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                          

AdaBoostClassifier processing ,,,
AdaBoostClassifier Score: 0.748 %


In [39]:
RF.fit(X_train, Y_train)
RF_feature = RF.feature_importances_
#RF_feature

rf_score = RF.score(X_test, Y_test)

print('RandomForestClassifier processing ,,,')
print('RandomForestClassifier Score: %.3f %%' % rf_score)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

RandomForestClassifier processing ,,,
RandomForestClassifier Score: 0.765 %


In [40]:
ET.fit(X_train, Y_train)
ET_feature = ET.feature_importances_
#ET_feature

et_score = ET.score(X_test, Y_test)

print('ExtraTreesClassifier processing ,,,')
print('ExtraTreeClassifier: %.3f %%' % et_score)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

ExtraTreesClassifier processing ,,,
ExtraTreeClassifier: 0.766 %


In [41]:
GB.fit(X_train, Y_train)

GB_feature = GB.feature_importances_
#GB_feature

gb_score = GB.score(X_test, Y_test)

print('GradientBoostingClassifier processing ,,,')
print('GradientBoostingClassifier Score: %.3f %%' % gb_score)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features='auto', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

GradientBoostingClassifier processing ,,,
GradientBoostingClassifier Score: 0.761 %


In [78]:
feature_df = pd.DataFrame({'features': X_train.columns.values, # names
                           'AdaBoost' : AB_feature,
                           'RandomForest' : RF_feature,
                           'ExtraTree' : ET_feature,
                           'GradientBoost' : GB_feature
                          })
#feature_df.features
feature_df.head(2)

,features,AdaBoost,RandomForest,ExtraTree,GradientBoost
0,duration,0.024841,0.003554,0.002528,0.002356
1,protocol_type,NaN,0.022990,0.041392,0.031850


In [81]:
n = 10
a_f = feature_df.nlargest(n, 'AdaBoost')
e_f = feature_df.nlargest(n, 'ExtraTree')
g_f = feature_df.nlargest(n, 'GradientBoost')
r_f = feature_df.nlargest(n, 'RandomForest')

result = pd.concat([a_f, e_f, g_f, r_f])
result = result.drop_duplicates() 
# result.shape
# print('\n')

# garbage = np.argsort(result.transpose().mean())
# garbage = garbage.sort_index()[:5] # FML... :'(
    
# garbage.index

# result = result.drop(garbage.index)
# result.shape



# arr = X_train.columns.to_numpy()
# result = result.set_index(np.take(arr,result.index))
result.in

,features,AdaBoost,RandomForest,ExtraTree,GradientBoost
23,dst_host_same_srv_rate,0.087415,0.050009,0.027385,0.003637
25,dst_host_same_src_port_rate,0.042108,0.037372,0.044946,0.007915
24,dst_host_diff_srv_rate,0.028493,0.028069,0.037521,0.052778
0,duration,0.024841,0.003554,0.002528,0.002356
29,dst_host_rerror_rate,0.017669,0.020824,0.032725,0.004649
12,count,0.017467,0.044294,0.040761,0.057363
5,wrong_fragment,0.011327,0.006679,0.015107,0.013814
18,same_srv_rate,0.010658,0.027390,0.025926,0.194059
9,num_file_creations,0.009409,0.000242,0.000254,0.001184
7,logged_in,0.007966,0.008731,0.064461,0.003302


### what the fuck am i doing with my life.... 45mins wasted on numpy 

In [82]:
# X_train_SF = X_train[result.index]
# X_test_SF = X_test[result.index]



selected_features = result['features'].values.tolist()
X_train_SF = X_train[selected_features]
X_test_SF = X_test[selected_features]


x = X_train_SF#.reshape(-1, 26)  # 31
y = Y_train['xAttack'].ravel()


# x=x[:20000]
# y=y[:20000]


x.shape
y.size

(125973, 21)

125973

In [83]:
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier() 
clf2 = RandomForestClassifier(n_estimators=25, random_state=1)# .77
clf3 = GradientBoostingClassifier() # .76
ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False) # .76 # without this lil fucker, Acc: 0.75 [Ensemble], 0.78 with 

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('et',ET)], voting='hard') 
# n =7 with better selection; .79
# n =7 ; .77
# n =10 ; .78
# n =14 ; .77


# eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('et',ET)], voting='soft') #.76

for clf, label in zip([clf1, clf2, clf3,ET, eclf], ['DecisionTreeClassifier', 'Random Forest', 'GradientBoostingClassifier','ExtraTreesClassifier', 'Ensemble']): 
    # scores = cross_val_score(clf, x, y, cv=2, scoring='accuracy') # cv= 5 
    # print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

    tmp = clf.fit(x,y)
    pred = clf.score(X_test_SF,Y_test)
    print("Acc: %0.2f [%s]" % (pred,label))

Acc: 0.74 [DecisionTreeClassifier]
Acc: 0.76 [Random Forest]
Acc: 0.72 [GradientBoostingClassifier]
Acc: 0.76 [ExtraTreesClassifier]
Acc: 0.76 [Ensemble]


## lets multithread this fucker

* https://stackoverflow.com/questions/42920148/using-sklearn-voting-ensemble-with-partial-fit

* https://gist.github.com/tomquisel/a421235422fdf6b51ec2ccc5e3dee1b4



In [0]:
# import multiprocessing

# ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False) # .76 
# RF = RandomForestClassifier(n_estimators=25, random_state=1)# .75
# GB = GradientBoostingClassifier() # .74
# ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False) # .77 # without this lil fucker, Acc: 0.75 [Ensemble]
# clfList = [ET,RF,GB,ET]

# def spawn(clf):
#   clf.fit(x,y)
#   print("Done another one!")

# import time
# start = time.time()

# if __name__ == '__main__':
  
#   for i in clfList:
#     print(i)
#     #spawn(i) # 16 secounds 

#     # # 16 secounds
#     # p = threading.Thread(target=spawn, args=(i,))
#     # p.start()
#     # p.join()
    
#   p=multiprocessing.Pool(6) # 15.65
#   results = p.map(spawn,clfList) # clfList has 4 models, first 2 are fast, last 2 are slow
#   results


# end = time.time()
# print(end - start)


In [0]:
# ET.score(X_test_SF,Y_test)


In [0]:
# from sklearn.cluster import KMeans
# for i in range(10):
#     tmp = KMeans(i)    
#     tmp = clf.fit(x,y)
#     tmp = tmp.score(X_test_SF,Y_test)
#     print(i,tmp)

